# Eigenvectors and Eigenvalues 


### Jacobi transformations

In [3]:
import torch
from torch.utils.cpp_extension import load

In [4]:
jacobi = load(name='jacobi',
             build_directory='./build',
             sources=['jacobi.cc'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             verbose=False)

In [19]:
print(torch.__config__.show())

PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70
  - CuDNN 7.6.5
  - Magma 2.5.2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=10.2, CUDNN_VERSION=7.6.5, CXX_COMPILER=/opt/rh/devtoolset-7/root/usr/bin/c++, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDEBUG -DUSE_KINETO -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_QNNPACK -DUSE_XNNPACK -DSYMBOLICATE_MOBILE_DEBUG_HANDLE -O2 -fPIC -Wno-narrowing -Wall -Wextra -Werror=return-type -Wno-missing-field-initi

Jacobi rotation for rows/columns $p$ and $q$:
$$
  P_{pq}(\phi) = \begin{pmatrix}
   1 &        &           &        &          &        &   \\
     & \cdots &           &        &          &        &   \\
     &        & \cos\phi  & \cdots & \sin\phi &        &   \\
     &        &  \vdots   &  1     &  \vdots  &        &   \\
     &        & -\sin\phi & \cdots & \cos\phi &        &   \\
     &        &           &        &          & \cdots &   \\
     &        &           &        &          &        & 1 \\
  \end{pmatrix}
$$

Let $A$ a real symmetric matrix, apply the Jacobi transformation:
$$
A' = P^T_{pq}(\phi) \cdot A \cdot P_{pq}(\phi)
$$


### Exercises
**(1)** Show that:
$$
a'_{pq} = (\cos^2\phi - \sin^2\phi)a_{pq} + (a_{pp} - a_{qq})\cos\phi\sin\phi
$$

**(2)** Requiring $a'_{pq}=0$ amounts to solve:
$$
\tan^2\phi + 2\theta\tan\phi-1 = 0
$$
where 
$$
\theta = \frac{a_{qq} - a_{pp}}{2 a_{pq}}
$$

**(3)** A solution $\phi_0 < \pi / 4$ is given by:
$$
\tan\phi_0 = \frac{\mathrm{sgn}(\theta)}{ \left|\theta\right| + \sqrt{\theta^2+1} }
$$

**(4)** Defining:
$$
S(A) = \sum_{r \neq s} \left|a_{rs}\right|^2, 
$$
show that for $\phi_0$:
$$
S(A') = S(A) - 2\left|a_{pq}\right|^2
$$

Applying iteratively Jacobi transformations, the above equation guarantees convergence to orthonormal eigenvectors:
$$
V = P_1 \cdot P_2 \cdots I
$$
with corresponding eigenvalues on the diagonal:
$$
D = V^T \cdot A \cdot V
$$



In [5]:
A_ = torch.randn((5,5)).double()
A = A_ + A_.t()
A

tensor([[ 0.8751, -1.9225,  0.2382, -0.8134, -2.1602],
        [-1.9225, -2.4920,  1.1830, -0.3045, -3.2797],
        [ 0.2382,  1.1830, -1.0930,  2.4268,  2.0069],
        [-0.8134, -0.3045,  2.4268,  0.1921, -1.3723],
        [-2.1602, -3.2797,  2.0069, -1.3723,  0.6081]], dtype=torch.float64)

In [22]:
A[0,0]

tensor(0.8751, dtype=torch.float64)

In [6]:
D, V = jacobi.symeig(A)

In [7]:
D

tensor([ 2.4896, -7.1738, -1.8946,  1.1758,  3.4931], dtype=torch.float64)

In [8]:
V

tensor([[ 0.3944,  0.3249,  0.2015,  0.6944, -0.4649],
        [-0.3741,  0.6137,  0.5237, -0.3883, -0.2415],
        [-0.4203, -0.4168,  0.6286,  0.4290,  0.2653],
        [-0.6826,  0.2932, -0.5348,  0.4026, -0.0046],
        [ 0.2488,  0.5080,  0.0629,  0.1446,  0.8094]], dtype=torch.float64)

In [9]:
Ds, Di = D.sort()
Ds

tensor([-7.1738, -1.8946,  1.1758,  2.4896,  3.4931], dtype=torch.float64)

In [10]:
Vs = V.t()[Di].t()
Vs

tensor([[ 0.3249,  0.2015,  0.6944,  0.3944, -0.4649],
        [ 0.6137,  0.5237, -0.3883, -0.3741, -0.2415],
        [-0.4168,  0.6286,  0.4290, -0.4203,  0.2653],
        [ 0.2932, -0.5348,  0.4026, -0.6826, -0.0046],
        [ 0.5080,  0.0629,  0.1446,  0.2488,  0.8094]], dtype=torch.float64)

In [12]:
torch.dist(A, V @ torch.diag_embed(D) @ V.t())

tensor(9.4534e-13, dtype=torch.float64)

In [14]:
Dmkl, Vmkl = torch.linalg.eigh(A)

In [15]:
torch.dist(Dmkl, Ds)

tensor(2.5704e-15, dtype=torch.float64)

### Exercises 
**(1)** Let $A$ and $B$ be a real symmetric matrices, with $B$ positive definite. Implement an efficient routine to solve the generalised eigenvalue problem:
$$
A \cdot x = \lambda B \cdot x
$$

**(2)** Implement a routine to that attempts to solve the non-linear eigenvalue problem:
$$
(\lambda^2 A + \lambda B + C) \cdot x = 0
$$
over the reals, for any compatible set of square matrices.
